In [1]:
from tensorflow.keras.optimizers import Adam
from keras.utils import Sequence
import pandas as pd
import numpy as np
import os

pi = 3.14159265359

maxval=1e9
minval=1e-9

2025-06-18 13:58:58.698197: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-18 13:58:59.677416: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from dataloaders.OptimizedDataGenerator_v2 import OptimizedDataGenerator
from models import *

In [3]:
model=CreateModel((16,16,20),n_filters=5,pool_size=3)
model.summary()

2025-06-18 13:59:02.172242: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3234 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 1g.5gb, pci bus id: 0000:81:00.0, compute capability: 8.0


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16, 16, 20)]      0         
                                                                 
 q_separable_conv2d (QSepar  (None, 14, 14, 5)         285       
 ableConv2D)                                                     
                                                                 
 q_activation (QActivation)  (None, 14, 14, 5)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 14, 14, 5)         30        
                                                                 
 q_activation_1 (QActivatio  (None, 14, 14, 5)         0         
 n)                                                              
                                                                 
 average_pooling2d (Average  (None, 4, 4, 5)           0     

In [4]:
# get best weights file
pitch = '50x12P5'
batch_size = 5000
fingerprint = '34c2da80'
timeslices = 20
files = os.listdir('/home/dajiang/smart-pixels-ml/weights/dataset_3src_16x16_weights/weights-{}-bs{}-{}-{}t-checkpoints'.format(pitch, batch_size, fingerprint, timeslices))

vlosses = [float(f.split("-v")[1].split(".hdf5")[0]) for f in files]
bestfile = files[np.argmin(vlosses)]
model.load_weights('/home/dajiang/smart-pixels-ml/weights/dataset_3src_16x16_weights/weights-{}-bs{}-{}-{}t-checkpoints/'.format(pitch, batch_size, fingerprint, timeslices)+bestfile)

print('Best model: {}'.format(bestfile))

Best model: weights.924-t-28193.48-v-30075.01.hdf5


In [6]:
# load in the test set
test_generator = OptimizedDataGenerator(
    load_from_tfrecords_dir = '/data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_{}_parquets/contained/TFR_files/{}t/TFR_val'.format(pitch, timeslices),
    quantize = True
)

Loading metadata from /data/dajiang/smart-pixels/dataset_3sr/shuffled/dataset_3sr_16x16_50x12P5_parquets/contained/TFR_files/20t/TFR_val/metadata.json


In [7]:
# predicts test data
p_test = model.predict(test_generator)

complete_truth = None
for _, y in test_generator:
    if complete_truth is None:
        complete_truth = y
    else:
        complete_truth = np.concatenate((complete_truth, y), axis=0)

# creates df with all predicted values and matrix elements - 4 predictions, all 10 unique matrix elements
df = pd.DataFrame(p_test,columns=['x','M11','y','M22','cotA','M33','cotB','M44','M21','M31','M32','M41','M42','M43'])

# stores all true values in same matrix as xtrue, ytrue, etc.
df['xtrue'] = complete_truth[:,0]
df['ytrue'] = complete_truth[:,1]
df['cotAtrue'] = complete_truth[:,2]
df['cotBtrue'] = complete_truth[:,3]
df['M11'] = minval+tf.math.maximum(df['M11'], 0)
df['M22'] = minval+tf.math.maximum(df['M22'], 0)
df['M33'] = minval+tf.math.maximum(df['M33'], 0)
df['M44'] = minval+tf.math.maximum(df['M44'], 0)

df['sigmax'] = abs(df['M11'])
df['sigmay'] = np.sqrt(df['M21']**2 + df['M22']**2)
df['sigmacotA'] = np.sqrt(df['M31']**2+df['M32']**2+df['M33']**2)
df['sigmacotB'] = np.sqrt(df['M41']**2+df['M42']**2+df['M43']**2+df['M44']**2)

# calculates residuals for x, y, cotA, cotB
df['residualsX'] = df['xtrue'] - df['x']
df['residualsY'] = df['ytrue'] - df['y']
df['residualsA'] = df['cotAtrue'] - df['cotA']
df['residualsB'] = df['cotBtrue'] - df['cotB']

# calculates pulls for x, y, cotA, cotB
df['pullx'] = (df['xtrue']-df['x'])/df['sigmax']
df['pully'] = (df['ytrue']-df['y'])/df['sigmay']
df['pullcotA'] = (df['cotAtrue']-df['cotA'])/df['sigmacotA']
df['pullcotB'] = (df['cotBtrue']-df['cotB'])/df['sigmacotB']

# stores results as parquet
df.to_parquet("/home/dajiang/smart-pixels-ml/processed_parquets/dataset_3src_16x16/test-dataset_3src_16x16/{}_bs{}_{}_{}t_train-dataset_3src_16x16_test-dataset_3src_16x16_vars.parquet".format(pitch, batch_size, fingerprint, timeslices))

 2/21 [=>............................] - ETA: 2s 

2025-06-18 13:59:57.432566: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-06-18 13:59:57.447733: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8906


21/21 [==============================] - 9s 444ms/step
